In [2]:
import os, sys

import pandas as pd
import polars as pl

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy

import sklearn
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import dproc, sgml, sgnn, sgpp, sgutil, custpp

print(sys.version)
for i in [pd, pl, mpl, sns, np, scipy, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        pass

2025-02-06 10:04:43.682733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738836283.694165   80173 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738836283.697685   80173 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 10:04:43.709327: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]
pandas 2.2.3
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
numpy 1.26.4
scipy 1.12.0
sklearn 1.5.2
xgboost 2.1.2
catboost 1.2.5


In [1]:
from itertools import combinations

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit, train_test_split
from sklearn.preprocessing import TargetEncoder
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [3]:
sc = sgutil.SGCache('img', 'result')
ss = ShuffleSplit(n_splits = 1, random_state = 123)
kf = KFold(5, random_state = 123, shuffle=True)

files = {
    'train': os.path.join('data', 'train.csv'),
    'train_extra': os.path.join('data', 'training_extra.csv'),
    'test': os.path.join('data', 'test.csv'),
}

t = sc.cache_result(
    'pipeline_2',
    lambda : make_pipeline(
        sgpp.PolarsProcessor(), 
        sgpp.ExprProcessor({
            'Compartments_c' : pl.col('Compartments').cast(pl.String).cast(pl.Categorical)
        }),
        sgpp.PandasCoverter(index_col = 'id'),
        sgpp.ApplyWrapper(
            sgpp.CatArrangerFreq(1, na_value = 'Unknown'),
            ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
        ), 
        custpp.WeightCapacityProcessor()
    ).fit(files['train']),
    rerun = 1
)
df_train = pd.concat(
    [t.transform(files['train']), t.transform(files['train_extra'])], axis = 0
)
df_test = t.transform(files['test'])

target = 'Price'
X_cat = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Compartments_c', 'wc_i']
X_num = ['Compartments', 'Weight Capacity (kg)']

In [5]:
from sklearn.base import TransformerMixin
class CatArrangerFreqNear(TransformerMixin):
    def __init__(self, min_frequency, na_value = None):
        self.min_frequency = min_frequency
        self.na_value = na_value

    def fit(self, X, y = None):
        if self.na_value is not None:
            c = {
                i: [self.na_value]
                for i in X.columns
            }
        else:
            c = {i: [] for i in X.columns}

        if self.min_frequency > 0:
            c = {
                i: c[i] + X[i].value_counts().pipe(lambda x: x.loc[(x >= self.min_frequency) & ~x.index.isin(c[i])]).index.tolist()
                for i in X.columns
            }
        else:
            c = {
                i: c[i] + X[i].loc[~X[i].isin(c[i])].unique().tolist() for i in X.columns
            }
        self.c_types_ = {
            i: pd.CategoricalDtype(c[i]) for i in X.columns
        }
        return self
        
    def transform(self, X):
        if self.na_value is not None:
            ret = pd.concat([
                dproc.rearrange_cat(X[k], v, lambda d, c: np.argmin(d - c) if c not in d else c, use_set = False).rename(k)
                for k, v in self.c_types_.items()
            ], axis = 1)
            if self.na_value is not None:
                return ret.fillna(self.na_value)
            return ret
        return X

    def get_params(self, deep=True):
        return {
            "min_frequency": self.min_frequency, 
            "unknown_value": self.unknown_value,
            "na_value": self.na_value
        }

    def set_output(self, transform='pandas'):
        pass

    def get_feature_names_out(self, X = None):
        return list(self.c_types_)

In [13]:
X = ['Weight Capacity (kg)']
cv = kf
tgt = TargetEncoder()
tgt_list = list()
cafn = sgml.CatArrangerFreqNear(30, df_train['Weight Capacity (kg)'].mode()[0])
cafn.fit(df_train[X].astype('category'))
for train_idx, valid_idx in cv.split(df_train[X], df_train[target]):
    df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    df_cv_train, df_valid = df_cv_train, df_valid
    tgt.fit(cafn.transform(df_cv_train[X].astype('category')) , df_cv_train[target])
    tgt_list.append(
        pd.Series(tgt.transform(
            cafn.transform(df_valid[X].astype('category'))
        )[:, 0], index = df_valid.index, name = '_'.join(X))
    )
prd = pd.concat(tgt_list)
root_mean_squared_error(
    df_train[target], prd.sort_index()
)

38.832717188489035